In [ ]:
# Convert to YOLO format
# Fork from: https://signate.jp/competitions/256/discussions/tutorial-how-to-convert-labels-from-multiple-jsons-to-a-single-txt-file
import os
import json
import random

In [ ]:
# Get path
path_labels    = '../data/train_annotations'      
path_images    = '../data/train_images'
path_out_file  = '../data/yolo_format_training.txt'

# Define classes as array
classes = ['Car', 'Pedestrian', 'Truck', 'Signal', 'Signs', 'Bicycle', 'Motorbike', 'Bus', 'Svehicle', 'Train']

# Image size:
WITH = 1936
HEIGHT = 1216

In [ ]:
# Yolo format (https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects)
def normalize_bbox(annot_bbox):
    """Normalize position over image size."""
    annot_bbox[0] = float(annot_bbox[0] / WITH)
    annot_bbox[1] = float(annot_bbox[1] / HEIGHT)
    annot_bbox[2] = float(annot_bbox[2] / WITH)
    annot_bbox[3] = float(annot_bbox[3] / HEIGHT)
    return(annot_bbox)

def to_yolo_bbox(annot_bbox):
    """Convert XML bbox to yolo format."""
    normalized_bbox = normalize_bbox(annot_bbox)
    yolo_w = normalized_bbox[2] - normalized_bbox[0]
    yolo_h = normalized_bbox[3] - normalized_bbox[1]
    X = normalized_bbox[0] + (yolo_w*0.5)
    Y = normalized_bbox[1] + (yolo_h*0.5)
    return([X, Y, yolo_h, yolo_h])

def generate_yolo_label(annot_bbox, classe):
    """Generate yolo format as string."""
    yolo_bbox = to_yolo_bbox(annot_bbox)
    label_to_write = str(classe) + ' ' + \
                        str(yolo_bbox[0]) + ' ' + \
                        str(yolo_bbox[1]) + ' ' + \
                        str(yolo_bbox[2]) + ' ' + \
                        str(yolo_bbox[3])
    return(label_to_write)

In [ ]:
out_file    = open(path_out_file, "w")
annotations = os.listdir(path_labels)

for i in range (0, len(annotations)): #here we browse all videos
    video_name = annotations[i].split('/')[-1].split('\\')[-1].split('.')[0]
    data       = json.load(open(os.path.join(path_labels, annotations[i])))
    print(video_name)
    
    for v in range (0,600): #here we browse all frames. Single movie has 600 frames
        img_name     = path_images+'/'+video_name+'/'+str(v)+".png"
        labels       = data['sequence'][v]
        str_to_write = img_name
        for c in range (0, len(classes)):
            try:
                for inst in data['sequence'][v][classes[c]]:
                    box           = inst['box2d']
                    str_to_write += ';' + generate_yolo_label(box, c)
            except Exception as e:
                continue #nothing, the class is just not presented in the frame
         
        if str_to_write != img_name: #we do not want to write images without annotations
            out_file.write(str_to_write+'\n')
        
out_file.close() 

In [ ]:
len(out_file)

### Prepare for training
- 1 image 1 file
- Split train/validation
- Yolo's training files (names, data, ...)

In [ ]:
def generate_yolo_labels(name, annots):
    """Generate label for individual image in YOLO format."""
    label_name = name.split('png')[0] + 'txt'
    label_file = open(label_name, 'w+')
    for annot in annots:
        label_file.write(annot + '\n')

    label_file.close()
    return(0)

In [ ]:
# Label file generation
list_image = []
with open(path_out_file, 'r') as yolo_labels:
    labels = yolo_labels.readline()
    while labels:
        # Get label content
        image_name = labels.split(';')[0]
        annots = labels.split(";")[1:]
        
        # Generate individual label
        generate_yolo_labels(image_name, annots)

        # Add name to list
        list_image.append(image_name)
        labels = yolo_labels.readline()

In [ ]:
# Split train-validation
nb_images = len(list_image)
ratio = 0.2
validation_size = int(ratio * nb_images)
valid_index = random.sample(range(nb_images), k=validation_size)

## Split images_set
train_set = []
valid_set = []
index = 0

for image in list_image:
    if index in valid_index:
        valid_set.append(image)
    else:
        train_set.append(image)
    index += 1
    
## Display
print("Dataset images numbers {} ({} datasets)".format(nb_images, int(nb_images/600)))
print("Train: {} / Valid {} images.".format(len(train_set), len(valid_set)))

In [ ]:
# Generata YOLO file for training
#path_relatif to darknet (src/darknet)
darknet_relatif = "../"

# train_file
with open('signate_train.txt', 'w+') as train_file:
    for img in train_set:
        train_file.write(darknet_relatif + img + '\n')

train_file.close()

# valid_file
with open('signate_val.txt', 'w+') as val_file:
    for img in valid_set:
        val_file.write(darknet_relatif + img + '\n')

val_file.close()

In [ ]:
# Modify yolo head and training parameters:
filters = (len(classes) + 5) * 3
max_batch = 2000*len(classes)
steps = (max_batch*0.8, max_batch*0.9)
print("Change in yolov4.cfg: classes={} / filters={}".format(len(classes), filters))
print("Recommended batch: {} with steps {}".format(max_batch, steps))

### Run training wiht
```shell
./darknet detector train signate_training/signate.data signate_training/yolov4-signate.cfg yolov4.conv.137 --map > signate_training/train.log
```